# Severity of storms 

### 1.1 Business problem


Our project will focus on analyzing and predicting the severity of tornadoes across various regions in America in terms of property damage. To analyze this, we will take the last 10 years of tornado data to represent an approximate measure of it. To calculate this, we will establish a threshold for measuring the severity using the amount of property damage caused by tornadoes. We will predict the property damage of tornadoes and use a scale of low, medium, and high to represent the severity.  
In addition, we will also compare attitudes across each of the 4 seasons - summer, spring, winter, fall, and see if there are any trends present across the 10 year span that shows when tornadoes are most common and damaging. The trends will help us be more accurate in predicting the property damage caused by the tornadoes  per region during different seasons/times of the year.


### 1.2 Datasets


Our dataset details instances of severe weather across a 15 year period. We will be looking at Tornados across this time period. We are given the locations that the Tornados are in, the category of Tornadoes, fatalities, property damage, and the length of the storm. From this we created a target feature called severity which assigns tornados a number from 1 to 10 based on how severe they are.


### 1.3 Proposed analytics solution

 How we get to the target variable -- severity index
The severity index will be calculated by taking into account the property damage, the lives lost in the event, 


In [ ]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## Read all data into one single dataframe
df_all_data = pd.read_csv('./dataset/storm_event_details_2010.csv')


for i in range(2011,2021):
    df_temp = pd.read_csv(f'./dataset/storm_event_details_{i}.csv')
    df_all_data = df_all_data.append(df_temp, ignore_index=True)



In [2]:
## Remove unused columns and format continuous columns

df_hur = df_all_data[df_all_data['EVENT_TYPE']=='Tornado']
df_hur = df_hur.drop(columns=['TOR_OTHER_WFO', 'END_YEARMONTH', 'EVENT_TYPE', 'END_DATE_TIME',
                                           'TOR_OTHER_CZ_STATE','TOR_OTHER_CZ_FIPS','TOR_OTHER_CZ_NAME','DATA_SOURCE','EPISODE_NARRATIVE',
                                            'EVENT_NARRATIVE','WFO','SOURCE','CZ_TIMEZONE','BEGIN_AZIMUTH','END_AZIMUTH','BEGIN_LAT',
                                            'END_LAT','BEGIN_LON','END_LON','STATE_FIPS','BEGIN_RANGE','END_RANGE','DAMAGE_CROPS',
                                            'BEGIN_TIME','END_TIME','BEGIN_LOCATION','END_LOCATION','FLOOD_CAUSE','MAGNITUDE_TYPE',
                                            'MAGNITUDE','CZ_FIPS','CZ_TYPE','CZ_NAME','CATEGORY'])
cols = ['INJURIES_INDIRECT', 'INJURIES_DIRECT', 'DEATHS_INDIRECT', 'DEATHS_DIRECT']
df_hur = df_hur.assign(HARM_TOTAL=df_hur[cols].sum(1)).drop(cols,1)
df_hur['TOR_AREA'] = df_hur['TOR_LENGTH']*df_hur['TOR_WIDTH']
df_hur = df_hur.drop(columns=['TOR_LENGTH', 'TOR_WIDTH'])
df_hur = df_hur.dropna()


/var/folders/b1/t1n97j495rq26g0zhqy_3ty80000gn/T/ipykernel_20737/2115226861.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df_hur = df_hur.assign(HARM_TOTAL=df_hur[cols].sum(1)).drop(cols,1)


In [3]:
## Format DAMAGE_PROPERTY column to be float instead of object

dmg = pd.DataFrame(df_hur['DAMAGE_PROPERTY'])

print(dmg)
for index, row in dmg.iterrows():
    val = row['DAMAGE_PROPERTY']
    if val[-1:] == 'B':
        row['DAMAGE_PROPERTY'] = float(val[:-1])*1000000
        # print(float(row['DAMAGE_PROPERTY'][:-1])*1000000)
    elif val[-1:] == 'M':
        row['DAMAGE_PROPERTY'] = float(val[:-1])*1000
        # print(float(row['DAMAGE_PROPERTY'][:-1])*1000)
    elif val[-1:] == 'K':
        row['DAMAGE_PROPERTY'] = float(val[:-1])*1
        # print(float(row['DAMAGE_PROPERTY'][:-1]))

df_hur['DAMAGE_PROPERTY'] = dmg

       DAMAGE_PROPERTY
75               5.00K
304              1.50M
617             10.00K
731            750.00K
732             10.00K
...                ...
688471           0.00K
688472         250.00K
688495          60.00K
688497           0.00K
688560           0.00K

[12945 rows x 1 columns]


In [4]:
## Sort by priority variable and find data split percentages

df_hur = df_hur.sort_values('DAMAGE_PROPERTY', ascending=False)
df_hur.info()
df_hur = df_hur.loc[df_hur['TOR_F_SCALE']!='EFU']
print(df_hur['TOR_F_SCALE'].value_counts()/len(df_hur) * 100)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12945 entries, 100619 to 688560
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BEGIN_YEARMONTH  12945 non-null  int64  
 1   BEGIN_DAY        12945 non-null  int64  
 2   END_DAY          12945 non-null  int64  
 3   EPISODE_ID       12945 non-null  int64  
 4   EVENT_ID         12945 non-null  int64  
 5   STATE            12945 non-null  object 
 6   YEAR             12945 non-null  int64  
 7   MONTH_NAME       12945 non-null  object 
 8   BEGIN_DATE_TIME  12945 non-null  object 
 9   DAMAGE_PROPERTY  12945 non-null  object 
 10  TOR_F_SCALE      12945 non-null  object 
 11  HARM_TOTAL       12945 non-null  int64  
 12  TOR_AREA         12945 non-null  float64
dtypes: float64(1), int64(7), object(5)
memory usage: 1.4+ MB
EF0    51.117296
EF1    35.737575
EF2     9.852883
EF3     2.576541
EF4     0.628231
EF5     0.087475
Name: TOR_F_SCALE, dtype: float

In [5]:
df_hur['MONTH_NAME'].value_counts()/len(df_hur) * 100

April        21.677932
May          20.548708
June         12.413519
July          7.093439
March         6.727634
August        6.067594
October       5.137177
November      4.580517
January       4.413519
February      4.159046
September     3.618290
December      3.562624
Name: MONTH_NAME, dtype: float64

In [6]:
## Split data into strata and sample proportinally (stratified sampling)

sampled_df = df_hur.groupby('TOR_F_SCALE', group_keys=False).apply(lambda x: x.sample(frac=0.1))
sampled_df
# sampled_df['MONTH_NAME'].value_counts()/len(df_hur) * 100

,BEGIN_YEARMONTH,BEGIN_DAY,END_DAY,EPISODE_ID,EVENT_ID,STATE,YEAR,MONTH_NAME,BEGIN_DATE_TIME,DAMAGE_PROPERTY,TOR_F_SCALE,HARM_TOTAL,TOR_AREA
455382,201705,1,1,116614,701343,GEORGIA,2017,May,01-MAY-17 09:24:00,25.0,EF0,0,354.0
530870,201808,28,28,128692,771917,WISCONSIN,2018,August,28-AUG-18 15:33:00,0.0,EF0,0,0.5
513393,201804,14,14,124296,748357,LOUISIANA,2018,April,14-APR-18 02:36:00,50.0,EF0,0,884.0
460629,201702,28,28,113282,678847,KENTUCKY,2017,February,28-FEB-17 22:35:00,35.0,EF0,0,121.8
47739,201006,30,30,42657,248773,TEXAS,2010,June,30-JUN-10 13:30:00,0.0,EF0,0,4.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
170153,201203,2,2,60205,359912,INDIANA,2012,March,02-MAR-12 15:09:00,55000.0,EF4,1,11697.0
662554,202007,8,8,148227,898306,MINNESOTA,2020,July,08-JUL-20 16:08:00,100.0,EF4,0,1534.0
344213,201512,26,26,101364,606471,TEXAS,2015,December,26-DEC-15 18:46:00,26000.0,EF4,478,5065.5
35891,201004,24,24,38376,224552,MISSISSIPPI,2010,April,24-APR-10 11:33:00,60000.0,EF4,41,55250.0


In [7]:
# visualize damage property 
#visualize tornado area 

### Example Data Quality Report for Continuous Variables
| Feature | Desc. | Count | % of Missing | Card. | Min. | Q1 | Median | Q3 | Max. | Mean | Std. Dev. | Notes |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| AAGE | Age | 199,523 | 0 | 91 | 0 | 15 | 33 | 50 | 90 | 34.49 | 22.31 |  |

### Example Data Quality Report for Categorical Variables
| Feature | Desc. | Count | % of Missing | Card. | Mode | Mode Freq. | Mode % | 2nd Mode | 2nd Mode Freq. | 2nd Mode Perc | Notes |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| ACLSWKR | Class of worker | 199,523 | 0 | 9 | Not in Universe | 100,245 | 50.24 | Private | 72,028 |  36.10 |  |

### Missing values and outliers

For missing values we will be dropping the instances because we will be taking a 10% sample of the instances. for outliers if its skewed then the outliers are more important. 

### Normalization

what we need to normalize and how we will normalize it

### Transformation